In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.graph_objs import Scatter, Layout, Figure, Data, Stream, YAxis, Marker
from plotly.offline import init_notebook_mode, plot
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

/home/AMathur/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [2]:
df = pd.read_csv('Spring 2018 - Product Case Data.csv')

In [3]:
df.head(2)

,Date,Channel,User Type,Land,Bounce,Purchase,Visitors_Control,Visitors_Variant,Unnamed: 8,Unnamed: 9
0,2014-10-10,Affiliate,Returning User,0,0,0,1211,1175,NaN,NaN
1,2014-10-10,Affiliate,Returning User,1,0,0,4076,4810,NaN,NaN


# Analysis of overall data

## Control

### Conversion rate

#### Landing on home page

#### # of visitors to the control home page that subsequently make a purchase (A)

In [128]:
df_land_purchase = df[(df['Land'] == 1) & (df['Purchase'] == 1)]

In [129]:
total_control_visitors = df_land_purchase['Visitors_Control'].sum()
total_control_visitors

131426

#### # of visitors to the control home page (B)

In [130]:
visitor_land_sum = df[df['Land'] == 1]['Visitors_Control'].sum()
visitor_land_sum

2604275

#### Aggregate conversion rate of control (A/B)

In [131]:
con_per_control = np.round(((total_control_visitors / visitor_land_sum ) * 100), 2)

In [132]:
con_per_control

5.05

#### Landing on other page

#### # of visitors to the control other page that subsequently make a purchase (A)

In [133]:
df_land_purchase = df[(df['Land'] == 0) & (df['Purchase'] == 1)]

In [134]:
total_control_visitors = df_land_purchase['Visitors_Control'].sum()
total_control_visitors

35117

#### # of visitors to the control other page (B)

In [135]:
visitor_land_sum = df[df['Land'] == 0]['Visitors_Control'].sum()
visitor_land_sum

392062

#### Aggregate conversion rate of control (A/B)

In [136]:
con_per_control = np.round(((total_control_visitors / visitor_land_sum ) * 100), 2)

In [137]:
con_per_control

8.96

### Bounce rate

#### # of control visitors who bounce (E)

In [9]:
control_visitor_bounce_sum = df[df['Bounce'] == 1]['Visitors_Control'].sum()
control_visitor_bounce_sum

1032963

#### Aggregate bounce rate of control (E/B)

In [10]:
print(f' Aggregate Bounce rate of control {np.round(((control_visitor_bounce_sum / visitor_land_sum ) * 100), 2)} %')

 Aggregate Bounce rate of control 39.66 %


## Variant

### Conversion Rate

#### Landing on Home Page

#### # of visitors to the variant home page that subsequently make a purchase (C)

In [11]:
df_land_purchase = df[(df['Land'] == 1) & (df['Purchase'] == 1)]

In [12]:
total_variant_visitors = df_land_purchase['Visitors_Variant'].sum()
total_variant_visitors

127655

#### # of visitors to the variant home page (D)

In [13]:
variant_visitor_land_sum = df[df['Land'] == 1]['Visitors_Variant'].sum()
variant_visitor_land_sum

2652133

#### Aggregate conversion rate of control (C/D)

In [14]:
con_per_variant = np.round(((total_variant_visitors / variant_visitor_land_sum ) * 100), 2)

In [15]:
con_per_variant

4.81

#### Landing on Home Page

#### # of visitors to the variant home page that subsequently make a purchase (C)

In [138]:
df_land_purchase = df[(df['Land'] == 0) & (df['Purchase'] == 1)]

In [139]:
total_variant_visitors = df_land_purchase['Visitors_Variant'].sum()
total_variant_visitors

33931

#### # of visitors to the variant home page (D)

In [140]:
variant_visitor_land_sum = df[df['Land'] == 0]['Visitors_Variant'].sum()
variant_visitor_land_sum

393720

#### Aggregate conversion rate of control (C/D)

In [141]:
con_per_variant = np.round(((total_variant_visitors / variant_visitor_land_sum ) * 100), 2)

In [142]:
con_per_variant

8.62

### Bounce Rate

#### # of variant visitors who bounce (F)

In [16]:
variant_visitor_bounce_sum = df[df['Bounce'] == 1]['Visitors_Variant'].sum()
variant_visitor_bounce_sum

1094416

#### Aggregate Bounce rate of Variant (F/D)

In [17]:
print(f' Aggregate Bounce rate of control {np.round(((variant_visitor_bounce_sum / variant_visitor_land_sum ) * 100), 2)} %')

 Aggregate Bounce rate of control 41.27 %


## Daily Analysis

### Conversion Rate

#### Landing on Home Page

#### # of visitors to the control and variant home page that subsequently make a purchase (A & C)

In [20]:
df_daily_purchase = df[(df['Land'] == 1) & (df['Purchase'] == 1)].groupby(['Date'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Purchase_Count', 'Visitors_Variant': 'Variant_Purchase_Count'})

#### # total control and variant who land on home page (B & D)


In [21]:
df_daily_land = df[(df['Land'] == 1)].groupby(['Date'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [22]:
df_daily = pd.merge(df_daily_purchase, df_daily_land, on='Date')

#### # Conversion rate - Control (A/B) & Variant (C/D)

In [23]:
df_daily['Control_Conversion_Rate'] = (df_daily['Control_Purchase_Count']/df_daily['Control_Land_Count'])*100
df_daily['Variant_Conversion_Rate'] = (df_daily['Variant_Purchase_Count']/df_daily['Variant_Land_Count'])*100

In [27]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily['Date'], y=df_daily['Control_Conversion_Rate'],
                    mode='lines+markers',
                    name='Control Daily Conversion rate'))
fig.add_trace(go.Scatter(x=df_daily['Date'], y=df_daily['Variant_Conversion_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Conversion rate'))


fig.update_layout(
        title = 'Conversion Rate trend of the control and variant by day'
)
fig

#### Landing on Other Page

#### # of visitors to the control and variant home page that subsequently make a purchase (A & C)

In [123]:
df_daily_purchase = df[(df['Land'] == 0) & (df['Purchase'] == 1)].groupby(['Date'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Purchase_Count', 'Visitors_Variant': 'Variant_Purchase_Count'})

#### # total control and variant who land on home page (B & D)


In [124]:
df_daily_land = df[(df['Land'] == 0)].groupby(['Date'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [125]:
df_daily = pd.merge(df_daily_purchase, df_daily_land, on='Date')

#### # Conversion rate - Control (A/B) & Variant (C/D)

In [126]:
df_daily['Control_Conversion_Rate'] = (df_daily['Control_Purchase_Count']/df_daily['Control_Land_Count'])*100
df_daily['Variant_Conversion_Rate'] = (df_daily['Variant_Purchase_Count']/df_daily['Variant_Land_Count'])*100

In [127]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily['Date'], y=df_daily['Control_Conversion_Rate'],
                    mode='lines+markers',
                    name='Control Daily Conversion rate'))
fig.add_trace(go.Scatter(x=df_daily['Date'], y=df_daily['Variant_Conversion_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Conversion rate'))


fig.update_layout(
        title = 'Conversion Rate trend of the control and variant by day'
)
fig

### Bounce Rate

#### # of visitors to the control and variant home page that bounce (E & F)

In [41]:
df_daily_bounce = df[df['Bounce']==1].groupby(['Date'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Bounce_Count', 'Visitors_Variant': 'Variant_Bounce_Count'})

#### # total control and variant who land on home page (B & D)

In [42]:
df_daily_land = df[(df['Land'] == 1)].groupby(['Date'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [43]:
df_daily_br = pd.merge(df_daily_bounce, df_daily_land, on='Date')

#### Bounce Rate - Control (A/B) & Variant (C/D)

In [44]:
df_daily_br['Control_Bounce_Rate'] = (df_daily_br['Control_Bounce_Count']/df_daily_br['Control_Land_Count'])*100
df_daily_br['Variant_Bounce_Rate'] = (df_daily_br['Variant_Bounce_Count']/df_daily_br['Variant_Land_Count'])*100

In [45]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily_br['Date'], y=df_daily_br['Control_Bounce_Rate'],
                    mode='lines+markers',
                    name='Control Daily Bounce rate'))
fig.add_trace(go.Scatter(x=df_daily_br['Date'], y=df_daily_br['Variant_Bounce_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Bounce rate'))


fig.update_layout(
        title = 'Bounce Rate trend of the control and variant by day'
)
fig

# Analysis of Deviation from Total (irrespective of design)

In [50]:
df_total = df.copy()

In [51]:
df_total['Visitors'] = df_total['Visitors_Control'] + df_total['Visitors_Variant']

## Conversion Rate

#### Landing on Home Page

In [57]:
df_total_land_purchase = df_total[(df_total['Land'] == 1) & (df_total['Purchase'] == 1)]

In [58]:
total_pur_visitors = df_total_land_purchase['Visitors'].sum()

In [59]:
total_visitor_land_sum = df_total[df_total['Land'] == 1]['Visitors'].sum()

In [60]:
print(f' Aggregate Conversion rate {np.round(((total_pur_visitors / total_visitor_land_sum ) * 100), 2)} %')

 Aggregate Conversion rate 4.93 %


#### Landing on Other page

In [114]:
df_total_land_purchase = df_total[(df_total['Land'] == 0) & (df_total['Purchase'] == 1)]

In [115]:
total_pur_visitors = df_total_land_purchase['Visitors'].sum()

In [116]:
total_visitor_land_sum = df_total[df_total['Land'] == 0]['Visitors'].sum()

In [117]:
print(f' Aggregate Conversion rate {np.round(((total_pur_visitors / total_visitor_land_sum ) * 100), 2)} %')

 Aggregate Conversion rate 8.79 %


### Daily Conversion Rate

#### Landing on Home Page

In [64]:
df_total_daily_purchase = df_total[(df_total['Land'] == 1) & (df_total['Purchase'] == 1)].groupby(['Date'])['Visitors'].sum()\
.reset_index().rename(columns = {'Visitors': 'Visitors_Purchase_Count'})

In [65]:
df_total_daily_land = df_total[(df_total['Land'] == 1)].groupby(['Date'])['Visitors'].sum()\
.reset_index().rename(columns = {'Visitors': 'Visitors_Land_Count'})

In [66]:
df_total_daily_pur = pd.merge(df_total_daily_purchase, df_total_daily_land, on='Date')

In [69]:
df_total_daily_pur['Conversion_Rate'] = (df_total_daily_pur['Visitors_Purchase_Count']/df_total_daily_pur['Visitors_Land_Count'])*100

In [70]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_total_daily_pur['Date'], y=df_total_daily_pur['Conversion_Rate'],
                    mode='lines+markers',
                    name='Daily Conversion Rate'))


fig.update_layout(
        title = 'Conversion Rate trend of total visitors by day'
)
fig

#### Landing on Other Page

In [118]:
df_total_daily_purchase = df_total[(df_total['Land'] == 0) & (df_total['Purchase'] == 1)].groupby(['Date'])['Visitors'].sum()\
.reset_index().rename(columns = {'Visitors': 'Visitors_Purchase_Count'})

In [119]:
df_total_daily_land = df_total[(df_total['Land'] == 0)].groupby(['Date'])['Visitors'].sum()\
.reset_index().rename(columns = {'Visitors': 'Visitors_Land_Count'})

In [120]:
df_total_daily_pur = pd.merge(df_total_daily_purchase, df_total_daily_land, on='Date')

In [121]:
df_total_daily_pur['Conversion_Rate'] = (df_total_daily_pur['Visitors_Purchase_Count']/df_total_daily_pur['Visitors_Land_Count'])*100

In [122]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_total_daily_pur['Date'], y=df_total_daily_pur['Conversion_Rate'],
                    mode='lines+markers',
                    name='Daily Conversion Rate'))


fig.update_layout(
        title = 'Conversion Rate trend of total visitors by day'
)
fig

## Bounce Rate

In [71]:
df_total_land_bounce = df_total[df_total['Bounce'] == 1]

In [72]:
total_bounce_visitors = df_total_land_bounce['Visitors'].sum()

In [73]:
total_visitor_land_sum = df_total[df_total['Land'] == 1]['Visitors'].sum()

In [74]:
print(f' Aggregate Conversion rate {np.round(((total_bounce_visitors / total_visitor_land_sum ) * 100), 2)} %')

 Aggregate Conversion rate 40.47 %


### Daily Bounce Rate

In [77]:
df_total_daily_bounce = df_total[df_total['Bounce'] == 1].groupby(['Date'])['Visitors'].sum()\
.reset_index().rename(columns = {'Visitors': 'Visitors_Bounce_Count'})

In [78]:
df_total_daily_land = df_total[(df_total['Land'] == 1)].groupby(['Date'])['Visitors'].sum()\
.reset_index().rename(columns = {'Visitors': 'Visitors_Land_Count'})

In [80]:
df_total_daily_br = pd.merge(df_total_daily_bounce, df_total_daily_land, on='Date')

In [81]:
df_total_daily_br['Bounce_Rate'] = (df_total_daily_br['Visitors_Bounce_Count']/df_total_daily_br['Visitors_Land_Count'])*100

In [82]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_total_daily_br['Date'], y=df_total_daily_br['Bounce_Rate'],
                    mode='lines+markers',
                    name='Daily Conversion Rate'))


fig.update_layout(
        title = 'Conversion Rate trend of total visitors by day'
)
fig

# Analysis of Channel Type

## Conversion Rate

#### Landing on Home Page

In [94]:
df_daily_purchase = df[(df['Land'] == 1) & (df['Purchase'] == 1)].groupby(['Channel'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Purchase_Count', 'Visitors_Variant': 'Variant_Purchase_Count'})

In [95]:
df_daily_land = df[(df['Land'] == 1)].groupby(['Channel'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [96]:
df_daily = pd.merge(df_daily_purchase, df_daily_land, on='Channel')

In [97]:
df_daily['Control_Conversion_Rate'] = (df_daily['Control_Purchase_Count']/df_daily['Control_Land_Count'])*100
df_daily['Variant_Conversion_Rate'] = (df_daily['Variant_Purchase_Count']/df_daily['Variant_Land_Count'])*100

In [98]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily['Channel'], y=df_daily['Control_Conversion_Rate'],
                    mode='lines+markers',
                    name='Control Daily Conversion rate'))
fig.add_trace(go.Scatter(x=df_daily['Channel'], y=df_daily['Variant_Conversion_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Conversion rate'))


fig.update_layout(
        title = 'Conversion Rate trend of the control and variant by Channel on Home Page'
)
fig

#### Landing on Other Page

In [99]:
df_daily_purchase = df[(df['Land'] == 0) & (df['Purchase'] == 1)].groupby(['Channel'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Purchase_Count', 'Visitors_Variant': 'Variant_Purchase_Count'})

In [100]:
df_daily_land = df[(df['Land'] == 0)].groupby(['Channel'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [101]:
df_daily = pd.merge(df_daily_purchase, df_daily_land, on='Channel')

In [102]:
df_daily['Control_Conversion_Rate'] = (df_daily['Control_Purchase_Count']/df_daily['Control_Land_Count'])*100
df_daily['Variant_Conversion_Rate'] = (df_daily['Variant_Purchase_Count']/df_daily['Variant_Land_Count'])*100

In [103]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily['Channel'], y=df_daily['Control_Conversion_Rate'],
                    mode='lines+markers',
                    name='Control Daily Conversion rate'))
fig.add_trace(go.Scatter(x=df_daily['Channel'], y=df_daily['Variant_Conversion_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Conversion rate'))


fig.update_layout(
        title = 'Conversion Rate trend of the control and variant by Channel on Home Page'
)
fig

## Bounce Rate

In [83]:
df_daily_bounce = df[df['Bounce']==1].groupby(['Channel'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Bounce_Count', 'Visitors_Variant': 'Variant_Bounce_Count'})

In [84]:
df_daily_land = df[(df['Land'] == 1)].groupby(['Channel'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [85]:
df_daily_br = pd.merge(df_daily_bounce, df_daily_land, on='Channel')

In [86]:
df_daily_br['Control_Bounce_Rate'] = (df_daily_br['Control_Bounce_Count']/df_daily_br['Control_Land_Count'])*100
df_daily_br['Variant_Bounce_Rate'] = (df_daily_br['Variant_Bounce_Count']/df_daily_br['Variant_Land_Count'])*100

In [87]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily_br['Channel'], y=df_daily_br['Control_Bounce_Rate'],
                    mode='lines+markers',
                    name='Control Daily Bounce rate'))
fig.add_trace(go.Scatter(x=df_daily_br['Channel'], y=df_daily_br['Variant_Bounce_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Bounce rate'))


fig.update_layout(
        title = 'Bounce Rate trend of the control and variant by Channel'
)
fig

# Analysis of User Type

## Conversion Rate

#### Landing on Home Page

In [104]:
df_daily_purchase = df[(df['Land'] == 1) & (df['Purchase'] == 1)].groupby(['User Type'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Purchase_Count', 'Visitors_Variant': 'Variant_Purchase_Count'})

In [105]:
df_daily_land = df[(df['Land'] == 1)].groupby(['User Type'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [106]:
df_daily = pd.merge(df_daily_purchase, df_daily_land, on='User Type')

In [107]:
df_daily['Control_Conversion_Rate'] = (df_daily['Control_Purchase_Count']/df_daily['Control_Land_Count'])*100
df_daily['Variant_Conversion_Rate'] = (df_daily['Variant_Purchase_Count']/df_daily['Variant_Land_Count'])*100

In [108]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily['User Type'], y=df_daily['Control_Conversion_Rate'],
                    mode='lines+markers',
                    name='Control Daily Conversion rate'))
fig.add_trace(go.Scatter(x=df_daily['User Type'], y=df_daily['Variant_Conversion_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Conversion rate'))


fig.update_layout(
        title = 'Conversion Rate trend of the control and variant by Channel on Home Page'
)
fig

#### Landing on Other Page

In [109]:
df_daily_purchase = df[(df['Land'] == 0) & (df['Purchase'] == 1)].groupby(['User Type'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Purchase_Count', 'Visitors_Variant': 'Variant_Purchase_Count'})

In [110]:
df_daily_land = df[(df['Land'] == 0)].groupby(['User Type'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [111]:
df_daily = pd.merge(df_daily_purchase, df_daily_land, on='User Type')

In [112]:
df_daily['Control_Conversion_Rate'] = (df_daily['Control_Purchase_Count']/df_daily['Control_Land_Count'])*100
df_daily['Variant_Conversion_Rate'] = (df_daily['Variant_Purchase_Count']/df_daily['Variant_Land_Count'])*100

In [113]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily['User Type'], y=df_daily['Control_Conversion_Rate'],
                    mode='lines+markers',
                    name='Control Daily Conversion rate'))
fig.add_trace(go.Scatter(x=df_daily['User Type'], y=df_daily['Variant_Conversion_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Conversion rate'))


fig.update_layout(
        title = 'Conversion Rate trend of the control and variant by Channel on Home Page'
)
fig

## Bounce Rate

In [88]:
df_daily_bounce = df[df['Bounce']==1].groupby(['User Type'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Bounce_Count', 'Visitors_Variant': 'Variant_Bounce_Count'})

In [89]:
df_daily_land = df[(df['Land'] == 1)].groupby(['User Type'])['Visitors_Control', 'Visitors_Variant'].sum()\
.reset_index().rename(columns = {'Visitors_Control': 'Control_Land_Count', 'Visitors_Variant': 'Variant_Land_Count'})

In [90]:
df_daily_br = pd.merge(df_daily_bounce, df_daily_land, on='User Type')

In [91]:
df_daily_br['Control_Bounce_Rate'] = (df_daily_br['Control_Bounce_Count']/df_daily_br['Control_Land_Count'])*100
df_daily_br['Variant_Bounce_Rate'] = (df_daily_br['Variant_Bounce_Count']/df_daily_br['Variant_Land_Count'])*100

In [93]:
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_daily_br['User Type'], y=df_daily_br['Control_Bounce_Rate'],
                    mode='lines+markers',
                    name='Control Daily Bounce rate'))
fig.add_trace(go.Scatter(x=df_daily_br['User Type'], y=df_daily_br['Variant_Bounce_Rate'],
                    mode='lines+markers',
                    name='Variant Daily Bounce rate'))


fig.update_layout(
        title = 'Bounce Rate trend of the control and variant by User Type'
)
fig